In [8]:
using JuMP, Gurobi, XLSX

In [9]:
xf = XLSX.readxlsx("toy-stores.xlsx")

XLSXFile("toy-stores.xlsx") containing 1 Worksheet
            sheetname size          range        
-------------------------------------------------
               Sheet1 102x12        A1:L102      


In [ ]:
model = Model(solver=Gurobisolver())

In [ ]:
set_optimizer_attribute(model, "Presolve", 0)
set_optimizer_attribute(model, "Heuristics", 0)
set_optimizer_attribute(model, "Cuts", 0)

In [ ]:
# Variables
@variable(model, 0 <= x[1:100,1:24]  <= 1)
@variable(model, 0 <= y[1:24], Bin )

In [ ]:
# Compute costs C_ij
C = zeros(Float64, 100,24)


In [ ]:
# Objective function
@objective(model, sum(x*transpose(C)) + y*transpose(f))

In [ ]:
# Constraint (2)